In [ ]:
import sqlite3

import pprint
pp = pprint.PrettyPrinter(indent=2)

In [ ]:
con = sqlite3.connect("./../data/zotero.sqlite")
cursor = con.cursor()


### List all tables

In [ ]:
cursor.execute("""
    SELECT name FROM sqlite_master WHERE type='table';
""")
tables = cursor.fetchall()
tables.sort(key = lambda x: x[0])

In [ ]:
for i, name in enumerate(tables):
    print("{:2d}. {}".format(i+1, name[0]))

### Count entries in each table

In [ ]:
itemCounts = {}
for i, (tName, ) in enumerate(tables):
    cursor.execute(f"""
        SELECT COUNT() from {tName}
    """)
    tCount = cursor.fetchone()[0]
    itemCounts[tName] = tCount

for tName, count in itemCounts.items():
    print("{:25s} - {:8d}".format(tName, count))

### Read user info

In [ ]:
users = {}
cursor.execute("""
    SELECT * FROM users
""")

for (userId, userName) in cursor.fetchall():
    users[userId] = userName

for userId, userName in users.items():
    print("{:10d} - {:20s}".format(userId, userName))

### Read settings

In [ ]:
settings = {}
cursor.execute("""
    SELECT * FROM settings
""")

# cursor.fetchall()
for (setting, key, value) in cursor.fetchall():
    if setting not in settings:
        settings[setting] = {}
    settings[setting][key] = value

# for k, v in settings.items():
#     print("{:20s}: {:20s}".format(k, v))
pp.pprint(settings)

### Get all collections

In [ ]:
collections = {}

cursor.execute("""
    SELECT 
    collectionID, collectionName, parentCollectionID 
    FROM collections
""")

for id, name, parentId in cursor.fetchall():
    collections[id] = {
        "name": name,
        "parentId": parentId
    }

# Print all collections and their parent
for id, coll in collections.items():
    if coll['parentId']:
        parentName = collections[coll['parentId']]['name']
    else:
        parentName = None
    print("{:4d}. {:<30s}        {:<30s}".format(
        id, 
        coll['name'], 
        parentName if parentName else ""))

### Get number of items in each collection

In [ ]:
cursor.execute("""
    SELECT 
    collectionID, COUNT(*)
    FROM collectionItems
    GROUP BY collectionID
""")

for colID, count in cursor.fetchall():
    collections[colID]["count"] = count

# Print all collections and their parent
for id, coll in collections.items():
    try:
        if coll['parentId']:
            parentName = collections[coll['parentId']]['name']
        else:
            parentName = None
        print("{:4d}. {:<30s}    {:4d}        {:<30s}".format(
            id, 
            coll['name'], coll['count'],
            parentName if parentName else ""))
    except Exception as err:
        print("Exception for ID =", id, "    ", coll['name'], "    ", err)


### Read all field IDs

In [ ]:
cursor.execute("""
    SELECT * FROM fields
""")

fieldsMapping = {}
for idx, fieldName, _ in cursor.fetchall():
    fieldsMapping[idx] = fieldName

pp.pprint(fieldsMapping)